<a href="https://colab.research.google.com/github/Michel-p16/PDS-Project/blob/capstone_korbi/distilbert_multi_inclEvalHeat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#connect drive
from google.colab import drive
drive.mount('/content/drive')

# Pfad zum neuen Multi-Select-Dataset
dataset_path_multi = '/content/drive/My Drive/ColabData/final_multi_question_data.json'

# Laden des neuen Datensatzes
import json
with open(dataset_path_multi, 'r') as file:
    dataset_multi = json.load(file)

# Laden des single Datensatzes
dataset_path_single = '/content/drive/My Drive/ColabData/final_single_question_data.json'
with open(dataset_path_single, 'r') as file:
    dataset_single = json.load(file)


!pip install transformers datasets
#1. Daten laden + filtern (MULTI SELECT here)
import json
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support

import os

import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import torch
import torch.nn as nn

# Filtere das Dataset nach Fragen mit dem Typ "MULTI_SELECT"
filtered_dataset_multi = [example for example in dataset_multi if example["type"] == "MULTI_SELECT"]




Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12=

In [2]:
#2. Daten formatieren (diesmal inkl Fragen Einbezug für training) -> binäre vektoren durch MultiLabelBinarizer

from sklearn.preprocessing import MultiLabelBinarizer

# Labels direkt in Float32 konvertieren
def convert_to_multi_select_format(data, mlb):
    formatted_data = []

    for example in data:
        question = example["question"]
        answers = example["answers"]

        for answer in answers:
            text = answer.get("answer_text", "")
            labels = answer.get("answer_label", "").split(",")

            # Labels bereinigen
            labels = [label.strip() for label in labels]

            if labels:  # Falls Labels vorhanden sind
                label_vector = mlb.transform([labels])[0]  # Binärvektor
                label_vector = label_vector.astype(float)  # **Hier direkt in float konvertieren!**

                formatted_data.append({
                    "question": question,  # Frage bleibt erhalten
                    "text": text,  # Antwort bleibt erhalten
                    "labels": label_vector  # Labels nun als float
                })

    return formatted_data



# Alle eindeutigen Labels aus dem Datensatz sammeln
all_labels = set()
for example in filtered_dataset_multi:
    for answer in example["answers"]:
        labels = answer.get("answer_label", "").split(",")
        labels = [label.strip() for label in labels]  # Führende/nachfolgende Leerzeichen entfernen
        all_labels.update(labels)

# MultiLabelBinarizer initialisieren
multi_label_binarizer = MultiLabelBinarizer(classes=sorted(list(all_labels)))  # Sortiert für Konsistenz
multi_label_binarizer.fit([list(all_labels)])

# Daten formatieren
formatted_multi_dataset = convert_to_multi_select_format(filtered_dataset_multi, multi_label_binarizer)

# Überprüfung der Ergebnisse
print(f"Anzahl der formatierten Beispiele: {len(formatted_multi_dataset)}")
if formatted_multi_dataset:
    print(f"Beispiel: {formatted_multi_dataset[0]}")
print(f"Alle möglichen Labels: {multi_label_binarizer.classes_}")


Anzahl der formatierten Beispiele: 745
Beispiel: {'question': 'What are your product interests?', 'text': "I'm interested in improving data quality and generating business cards.", 'labels': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])}
Alle möglichen Labels: ["'Angelina Haug'" "'Domiki Stein'" "'Erik Schneider'" "'Jens Roschmann'"
 "'Jessica Hanke'" "'Joachim Wagner'" "'Johannes Wagner'" "'Marisa Peng'"
 "'Oliver Eibel'" "'Sandro Kalter'" "'Sean Kennin'" "'Stephan Maier'"
 "'Tim Persson'" '1 week' '100' '2 weeks' '200' '234' '256' '3 weeks'
 '300' 'AKW100' 'AX100' 'Automotive radar target simulation'
 'BusinessCards' 'Capture trade fair contacts' 'Clean up CRM' 'Competitor'
 'Data Cleansing' 'DataEnrichment' 'DataQuality'
 'Display port debugging and compliance' 'Double-Pulse Testing' 'Email'

In [3]:
#3. Daten splitten + formatieren in Hugging Face Dataset

#Split 80/20
train_data_multi_formatted, eval_data_multi_formatted = train_test_split(formatted_multi_dataset, test_size=0.2, random_state=42)

# Multi-Label-Trainings- und Evaluations-Dataset erstellen
train_dataset_multi = Dataset.from_pandas(pd.DataFrame(train_data_multi_formatted))
eval_dataset_multi = Dataset.from_pandas(pd.DataFrame(eval_data_multi_formatted))

# Überprüfung der Datenmengen
print(f"Anzahl der Trainingsdaten: {len(train_dataset_multi)}")
print(f"Anzahl der Evaluationsdaten: {len(eval_dataset_multi)}")

Anzahl der Trainingsdaten: 596
Anzahl der Evaluationsdaten: 149


In [4]:
#4. Dataset vorverarbeitung

from transformers import AutoTokenizer
import torch
from datasets import Dataset

# Tokenizer initialisieren
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Vorverarbeitungsfunktion für Multi-Label-Daten
def preprocess_function_multi(examples):
    return tokenizer(
        examples["question"],  # Frage
        examples["text"],      # Antwort
        padding="max_length",
        truncation=True,
        max_length=128  # Falls nötig, auf 128 erhöht
    )

# Tokenizer-Funktion an
train_dataset_multi = train_dataset_multi.map(preprocess_function_multi, batched=True)
eval_dataset_multi = eval_dataset_multi.map(preprocess_function_multi, batched=True)

# PyTorch-kompatibles Format setzen
train_dataset_multi.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
eval_dataset_multi.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Struktur überprüfen
print(train_dataset_multi[0])
print(train_dataset_multi[0]["labels"])  # Sollte ein Float-Tensor sein
print(train_dataset_multi[0]["labels"].dtype)  # Erwartet: torch.float32



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/596 [00:00<?, ? examples/s]

Map:   0%|          | 0/149 [00:00<?, ? examples/s]

{'labels': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'input_ids': tensor([  101,  2054,  2024,  2115,  4031,  5426,  1029,   102,  1045,  2342,
         1037,  6994,  2005,  4372, 13149,  2075,  2026,  8013,  2951,  1012,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0, 

In [5]:
#5. PyTorch-kompatibles Format setzen
train_dataset_multi.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
eval_dataset_multi.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


In [7]:
#6. Training

import pickle
from sklearn.metrics import precision_recall_fscore_support
from transformers import DistilBertForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
import torch.nn.functional as F
import torch

# Loss-Funktion für Multi-Label-Klassifikation
def custom_loss(predictions, labels):
    return F.binary_cross_entropy_with_logits(predictions, labels)

# Eigene Trainer-Klasse mit angepasster Loss-Funktion
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")  # Entferne Labels aus Inputs
        outputs = model(**inputs)      # Modell-Vorhersagen
        logits = outputs.logits        # Logits extrahieren
        loss = custom_loss(logits, labels)  # BCEWithLogitsLoss berechnen
        return (loss, outputs) if return_outputs else loss

# Modell initialisieren (Multi-Label-Modus)
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(multi_label_binarizer.classes_),  # Anzahl der Labels
    problem_type="multi_label_classification"  # WICHTIG für Multi-Label
)

# Speichern des Label-Mappings für späteres Decoding
label_mapping = {idx: label for idx, label in enumerate(multi_label_binarizer.classes_)}
inverse_label_mapping = {label: idx for idx, label in enumerate(multi_label_binarizer.classes_)}

label_mapping_path = "/content/drive/My Drive/label_mapping_multi_wQ.pkl"
with open(label_mapping_path, "wb") as file:
    pickle.dump(label_mapping, file)

print("Label-Mapping erfolgreich gespeichert.")

# Erweiterte compute_metrics-Funktion
metrics_log = []  #save list

# Anpassung der Metriken (für Multi-Label Klassifikation)
def compute_metrics(pred):
    labels = pred.label_ids  # Wahre Labels
    preds = torch.sigmoid(torch.tensor(pred.predictions)) > 0.5  # Wahrscheinlichkeiten -> Binärwerte

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds.numpy(), average="weighted"
    )
    acc = (preds.numpy() == labels).all(axis=1).mean()  # Beispielgenauigkeit

  # save metrics, losses im callback
    metrics_log.append({
        'epoch': len(metrics_log) + 1,
        'training_loss': None,  # Wird aus dem Callback gefüllt
        'validation_loss': None,  # Wird aus dem Callback gefüllt
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    })

    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall,
    }

# TrainingArgs
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=25,
    learning_rate=5e-5,
    weight_decay=0.01,  # tried .05 -> worse
    #adam_beta1=0.9, #senken falls langsam reaktion
    #adam_beta2=0.999, #senekn bei wenig lernprogress
    #adam_epsilon=1e-6,  # erhöhen falls instabil
    #label_smoothing_factor=0.1,  #bessere Generalisierung
    evaluation_strategy="epoch",  # eval pro epoch
    save_strategy="epoch",  # save pro epoch
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="none",  # ausgeschaltet, weil kein Zugriff zu repo
    logging_steps=10,  # Alle 10 Schritte loggen
)

# Early Stopping Callback
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=1,  # stop at 1 epoch worse
    early_stopping_threshold=0.000  #  min same
)

#loss add for csv
from transformers import TrainerCallback

class LossCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        """Speichert Training & Validation Loss"""
        if logs:
            if "loss" in logs and len(metrics_log) > 0:
                metrics_log[-1]["training_loss"] = logs["loss"]

            if "eval_loss" in logs and len(metrics_log) > 0:
                metrics_log[-1]["validation_loss"] = logs["eval_loss"]


# Trainer-Objekt erstellen
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_multi,
    eval_dataset=eval_dataset_multi,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping, LossCallback()],  # Loss Callback
)

# Training starten
trainer.train()

# Metriken speichern
metrics_df = pd.DataFrame(metrics_log)
metrics_csv_path = "/content/drive/My Drive/distilMulti_training_metrics.csv"
metrics_df.to_csv(metrics_csv_path, index=False)
print(f"Metriken wurden erfolgreich unter {metrics_csv_path} gespeichert.")

# Modell und Tokenizer speichern
save_path = "/content/drive/My Drive/multi_distilbert_wQ"
os.makedirs(save_path, exist_ok=True)  # Erstelle den Pfad, falls er nicht existiert
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print("Das Modell wurde erfolgreich gespeichert.")



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Label-Mapping erfolgreich gespeichert.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.155400,0.144671,0.000000,0.000000,0.000000,0.000000
2,0.105800,0.096253,0.053691,0.107475,0.118172,0.110368
3,0.078200,0.072442,0.127517,0.367418,0.450304,0.357860
4,0.064300,0.058009,0.241611,0.501351,0.560340,0.488294
5,0.054500,0.051477,0.322148,0.546684,0.688350,0.501672
6,0.039700,0.043218,0.416107,0.696597,0.753462,0.692308
7,0.030300,0.035290,0.583893,0.786696,0.836965,0.769231
8,0.030600,0.029538,0.644295,0.791799,0.824725,0.782609
9,0.022600,0.026515,0.671141,0.841609,0.911120,0.812709
10,0.015800,0.024476,0.718121,0.863325,0.940241,0.836120


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Metriken wurden erfolgreich unter /content/drive/My Drive/distilMulti_training_metrics.csv gespeichert.
Das Modell wurde erfolgreich gespeichert.


In [ ]:
#7. manueller test

import torch
import random
import pickle
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Modell & Tokenizer laden
model_path = "/content/drive/My Drive/multi_distilbert_wQ"
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Label Mapping laden
label_mapping_path = "/content/drive/My Drive/label_mapping_multi_wQ.pkl"
with open(label_mapping_path, "rb") as file:
    label_mapping = pickle.load(file)

# Fragen aus dem Dataset extrahieren
all_questions = list(set(example["question"] for example in dataset_multi))

# 5 Zufällige Frage auswählen
selected_questions = random.sample(all_questions, 5)

# Evaluation für jede Frage
for idx, question in enumerate(selected_questions, start=1):
    print(f"Frage {idx}: {question}")

    # Manuelle Eingabe der Antwort
    user_answer = input("Bitte geben Sie eine Antwort ein: ")

    # Eingabe tokenisieren (Frage + Antwort)
    inputs = tokenizer(
        question,  # Frage
        user_answer,  # Antwort
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=128
    )

    # Modellvorhersage (ohne Gradientenberechnung)
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits  # Rohwerte des Modells

    # Wahrscheinlichkeiten berechnen
    probs = torch.sigmoid(logits).squeeze().tolist()

    # Schwellenwert setzen (z.B. 0.5)
    threshold = 0.25
    predicted_labels_indices = [idx for idx, prob in enumerate(probs) if prob > threshold]

    # Vorhergesagte Labels mappen
    predicted_labels = [label_mapping[idx] for idx in predicted_labels_indices]

    # Ausgabe der Vorhersagen
    print("Predicted weighted vector (probabilities for each label):")
    print(probs)

    print("Predicted binary vector (thresholded):")
    binary_vector = [1 if prob > threshold else 0 for prob in probs]
    print(binary_vector)

    print("Predicted Labels:")
    print(predicted_labels if predicted_labels else "Keine Labels vorhergesagt")

    print("-" * 60)  # Trennlinie für bessere Übersicht


Frage 1: What type of contact is it?
Bitte geben Sie eine Antwort ein: Itis an follow up call with an existing contact.
Predicted weighted vector (probabilities for each label):
[0.0008787182159721851, 0.00041149588651023805, 0.00013020739424973726, 0.00010654117068042979, 0.0010093594901263714, 0.00037621031515300274, 0.0004055496247019619, 0.0005291283596307039, 0.0006268151919357479, 0.00026271678507328033, 0.0014424180844798684, 5.7042987464228645e-05, 0.00013852023403160274, 0.0005981409340165555, 5.3281066357158124e-05, 0.00017603067681193352, 0.00013979901268612593, 7.153775368351489e-05, 8.181713201338425e-05, 0.0003138036699965596, 0.00015143259952310473, 0.00010392850526841357, 7.468118565157056e-05, 0.00017522468988317996, 0.00021101439779158682, 2.2128158889245242e-05, 7.97323664301075e-05, 0.005123147740960121, 2.197862704633735e-05, 5.753028017352335e-05, 0.00035328074591234326, 0.0003703544498421252, 0.000998732983134687, 0.0008185191545635462, 0.2886880040168762, 2.6446

In [ ]:
# MULTI TESTING + HEATMAP works

# Installiere notwendige Pakete (falls nicht vorhanden)
!pip install ipywidgets transformers torch

import ipywidgets as widgets
from IPython.display import display
import torch
import random
import pickle
from torch.utils.tensorboard import SummaryWriter
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image
import numpy as np

# Liste mit möglichen Fragen (Beispiel)
all_questions = [
    "What type of contact is it?",
    "What problem are you trying to solve?",
    "Who should I CC on the follow-up?",
    "When would you like a follow-up?",
    "What follow-up is planned?",
    "What are you interested in?",
    "What products are you interested in?",
    "What are your product interests?",
    "What products interest you?"
]

# Funktion zum Zufällig Wählen einer Frage
def get_random_question():
    return random.choice(all_questions)

# 🌟 **Modell und Tokenizer für DistilBERT laden**
model_path_distilbert = "/content/drive/My Drive/multi_distilbert_wQ"
tokenizer_distilbert = AutoTokenizer.from_pretrained(model_path_distilbert)
model_distilbert = AutoModelForSequenceClassification.from_pretrained(model_path_distilbert)

# 🌟 **Modell und Tokenizer für Roberta laden**
model_path_roberta = "/content/drive/My Drive/multi_roberta"
tokenizer_roberta = AutoTokenizer.from_pretrained(model_path_roberta)
model_roberta = AutoModelForSequenceClassification.from_pretrained(model_path_roberta)

# 🌟 Funktion zur Erstellung einer Heatmap
def create_heatmap(probabilities, label_mapping, title="Heatmap"):
    """Erstellt eine Heatmap basierend auf den Wahrscheinlichkeiten und Labels."""
    plt.figure(figsize=(12, 4))
    plt.imshow([probabilities], cmap="YlGnBu", aspect="auto")
    plt.colorbar(label="Probability")
    plt.yticks([])
    plt.xticks(range(len(probabilities)), [label_mapping[i] for i in range(len(probabilities))], rotation=90)
    plt.title(title)

    # Heatmap als PNG speichern
    buf = BytesIO()
    plt.savefig(buf, format="png", bbox_inches="tight")
    buf.seek(0)
    image_bytes = buf.read()
    buf.close()
    plt.close()
    return image_bytes  # Rückgabe als Byte-Array

# 🌟 Anpassung der Vorhersagefunktion
def predict_label_multiple_models(user_answer, question):
    if not user_answer or not question:
        return "Please answer."

    # DistilBERT-Vorhersage
    inputs_distilbert = tokenizer_distilbert(
        question, user_answer,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=128
    )
    model_distilbert.eval()
    with torch.no_grad():
        outputs_distilbert = model_distilbert(**inputs_distilbert)
        logits_distilbert = outputs_distilbert.logits
        probabilities_distilbert = torch.sigmoid(logits_distilbert).squeeze().tolist()
        predicted_indices_distilbert = [idx for idx, prob in enumerate(probabilities_distilbert) if prob > 0.25]
        predicted_labels_distilbert = [label_mapping[idx] for idx in predicted_indices_distilbert]

    # Roberta-Vorhersage
    inputs_roberta = tokenizer_roberta(
        question, user_answer,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=128
    )
    model_roberta.eval()
    with torch.no_grad():
        outputs_roberta = model_roberta(**inputs_roberta)
        logits_roberta = outputs_roberta.logits
        probabilities_roberta = torch.sigmoid(logits_roberta).squeeze().tolist()
        predicted_indices_roberta = [idx for idx, prob in enumerate(probabilities_roberta) if prob > 0.25]
        predicted_labels_roberta = [label_mapping[idx] for idx in predicted_indices_roberta]

    # Ergebnisse zurückgeben
    return {
        "DistilBERT": predicted_labels_distilbert if predicted_labels_distilbert else "No Labels predicted",
        "Roberta": predicted_labels_roberta if predicted_labels_roberta else "No Labels predicted",
        "DistilBERT_probs": probabilities_distilbert,
        "Roberta_probs": probabilities_roberta
    }

# 🌟 Anpassung des Predict-Buttons
def on_predict(b):
    predictions = predict_label_multiple_models(answer_text.value, question_text.value)

    # Textausgabe für die Predictions
    output_label.value = (
        f"DistilBERT Prediction: {', '.join(predictions['DistilBERT'])}\n"
        f"Roberta Prediction: {', '.join(predictions['Roberta'])}"
    )

    # Erstelle Heatmaps für beide Modelle
    distilbert_heatmap_bytes = create_heatmap(predictions["DistilBERT_probs"], label_mapping, title="DistilBERT Heatmap")
    roberta_heatmap_bytes = create_heatmap(predictions["Roberta_probs"], label_mapping, title="Roberta Heatmap")

    # Konvertiere Heatmaps in widgets.Image
    distilbert_image_widget = widgets.Image(value=distilbert_heatmap_bytes, format="png", width=600, height=300)
    roberta_image_widget = widgets.Image(value=roberta_heatmap_bytes, format="png", width=600, height=300)

    # Aktualisiere die Heatmap-Ansicht nebeneinander
    with heatmap_output:
        heatmap_output.clear_output()
        display(widgets.HBox([distilbert_image_widget, roberta_image_widget]))

# 🌟 Funktion für den "Neue Frage"-Button
def on_new_question(b):
    question_text.value = get_random_question()
    answer_text.value = ""
    output_label.value = "Waiting for input..."
    with heatmap_output:
        heatmap_output.clear_output()

# 🌟 UI mit zusätzlichem Heatmap-Bereich
heatmap_output = widgets.Output()

question_text = widgets.Text(value=get_random_question(), description="Question:", disabled=False)
answer_text = widgets.Text(description="Answer:")
predict_button = widgets.Button(description="🔍 Predict")
new_question_button = widgets.Button(description="New Question")
output_label = widgets.Label(value="Waiting for input...")

predict_button.on_click(on_predict)
new_question_button.on_click(on_new_question)

display(widgets.VBox([
    question_text,
    answer_text,
    widgets.HBox([predict_button, new_question_button]),
    output_label,
    heatmap_output  # Heatmaps nebeneinander
]))


In [ ]:
model_path_distilbert_single = "/content/drive/My Drive/final_model"
model_path_distilbert_multi = "/content/drive/My Drive/multi_distilbert_wQ"
model_path_roberta_single = "/content/drive/My Drive/roberta_finetuned"
model_path_roberta_multi =  "/content/drive/My Drive/multi_roberta"


In [ ]:
#Mapper Datei single erzeugen+save für dashboard

import json
import pickle

# Pfad zur JSON-Datei
dataset_path_single = '/content/drive/My Drive/ColabData/final_single_question_data.json'

# Pfad zur Mapping-Datei
label_mapping_path_single = '/content/drive/My Drive/single_label_mapping.pkl'

# Laden des Datensatzes
with open(dataset_path_single, 'r') as file:
    dataset_single = json.load(file)

# Funktion zur Extraktion der Labels und Erstellung eines Mappings
def extract_and_map_labels(dataset):
    label_set = set()
    for item in dataset:
        for answer in item.get('answers', []):
            label_set.add(answer['answer_label'])
    # Erstelle ein Mapping von Index zu Label
    label_mapping = {idx: label for idx, label in enumerate(sorted(label_set))}
    return label_mapping

# Labels extrahieren und mappen
label_mapping = extract_and_map_labels(dataset_single)

# Mapping in eine Datei speichern
with open(label_mapping_path_single, 'wb') as file:
    pickle.dump(label_mapping, file)

# Überprüfen, ob die Datei korrekt gespeichert wurde
print(f"Label Mapping gespeichert unter: {label_mapping_path_single}")



Label Mapping gespeichert unter: /content/drive/My Drive/single_label_mapping.pkl


In [ ]:
print("Loaded Label Mapping:", label_mapping)

Loaded Label Mapping: {0: '1-10', 1: '1-5', 2: '11-15', 3: '11-50', 4: '16-20', 5: '201-2000', 6: '21-30', 7: '31-40', 8: '51-200', 9: '6-10', 10: 'Adito', 11: 'Aerospace', 12: 'Applicant', 13: 'Automotive', 14: 'CAS', 15: 'Call', 16: 'Close.io', 17: 'Computers & Networks', 18: 'Construction company', 19: 'Consultant, Planner, Architect', 20: 'Craft enterprises', 21: 'Defense', 22: 'Education sector', 23: 'End User', 24: 'English', 25: 'Existing customer', 26: 'German', 27: 'Government', 28: 'HubSpot', 29: 'Industrial', 30: 'Italian', 31: 'Japanese ', 32: 'Medical', 33: 'Meeting', 34: 'Microsoft Dynamics', 35: 'Network Operators & Infrastructure', 36: 'New customer', 37: 'No', 38: 'Offer', 39: 'Partner', 40: 'Physical Security', 41: 'Pipedrive', 42: 'Production company', 43: 'Public Safety / Law Enforcement', 44: 'R&D', 45: 'SAP Sales Cloud', 46: 'Salesforce', 47: 'Satisfied', 48: 'Scaffolding company', 49: 'Spanish', 50: 'Trading company', 51: 'Unsatisfied', 52: 'Very satisfied', 53: 

In [ ]:
# SINGLE MODEL TESTING + HEATMAP works

# Installiere notwendige Pakete (falls nicht vorhanden)
!pip install ipywidgets transformers torch

import ipywidgets as widgets
from IPython.display import display
import torch
import random
import pickle
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image
import numpy as np

# 🌟 **Fragen aus dem Single-Datensatz**
single_select_questions = [
    "What type of customer are you?",
    "How satisfied are you with our service?",
    "What's the average size of your trade fair team?",
    "What CRM system are you currently using?",
    "What are the next steps?",
    "What language should we use to communicate?",
    "What kind of company is this?",
    "How many people work at your company?",
    "Want email marketing updates?",
    "What industry are you in?",
    "Do you consent to the processing of your data?",
    "What is the specific customer group you're targeting?"
]

# Funktion zum Zufällig Wählen einer Frage
def get_random_question():
    return random.choice(single_select_questions)

# 🌟 **Pfad zu den Modellen**
model_path_distilbert_single = "/content/drive/My Drive/final_model"
model_path_roberta_single = "/content/drive/My Drive/roberta_finetuned"

# 🌟 **Modell und Tokenizer für DistilBERT laden**
tokenizer_distilbert = AutoTokenizer.from_pretrained(model_path_distilbert_single)
model_distilbert = AutoModelForSequenceClassification.from_pretrained(model_path_distilbert_single)

# 🌟 **Modell und Tokenizer für Roberta laden**
tokenizer_roberta = AutoTokenizer.from_pretrained(model_path_roberta_single)
model_roberta = AutoModelForSequenceClassification.from_pretrained(model_path_roberta_single)

# 🌟 Label-Mapping laden
label_mapping_path = '/content/drive/My Drive/single_label_mapping.pkl'
with open(label_mapping_path, 'rb') as file:
    label_mapping = pickle.load(file)

# 🌟 Funktion zur Erstellung einer Heatmap
def create_heatmap(probabilities, title="Heatmap"):
    plt.figure(figsize=(12, 4))

    # Sortiere die Labels basierend auf den Indizes
    labels = [label_mapping[idx] for idx in range(len(label_mapping))]

    plt.imshow([probabilities], cmap="YlGnBu", aspect="auto")
    plt.colorbar(label="Probability")
    plt.yticks([])  # Keine Y-Ticks anzeigen
    plt.xticks(range(len(probabilities)), labels, rotation=90)  # Labels als X-Ticks verwenden
    plt.title(title)

    # Heatmap als PNG speichern
    buf = BytesIO()
    plt.savefig(buf, format="png", bbox_inches="tight")
    buf.seek(0)
    image_bytes = buf.read()
    buf.close()
    plt.close()
    return image_bytes




# 🌟 Vorhersage-Funktion
def predict_answer(question, answer):
    predictions = {}
    heatmaps = {}

    # Modelle und Tokenizer
    models = {
        "distilbert_single": model_distilbert,
        "roberta_single": model_roberta
    }
    tokenizers = {
        "distilbert_single": tokenizer_distilbert,
        "roberta_single": tokenizer_roberta
    }

    for model_key in models.keys():
        tokenizer = tokenizers[model_key]
        model = models[model_key]
        inputs = tokenizer(question, answer, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
        model.eval()
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            probabilities = torch.sigmoid(logits).squeeze().tolist()  # Wahrscheinlichkeiten
            predicted_index = torch.argmax(logits, dim=1).item()  # Index der höchsten Wahrscheinlichkeit

            # Mapping anwenden
            label_name = label_mapping.get(predicted_index, f"Unknown ({predicted_index})")

            predictions[model_key] = label_name
            heatmaps[model_key] = create_heatmap(probabilities, title=f"{model_key} Heatmap")

    return predictions, heatmaps


# 🌟 Anpassung des Predict-Buttons
def on_predict(b):
    question = question_text.value
    answer = answer_text.value

    predictions, heatmaps = predict_answer(question, answer)

    # Ausgabe der Predictions
    output_label.value = (
        f"<b>DistilBERT Prediction:</b> {predictions['distilbert_single']}<br>"
        f"<b>Roberta Prediction:</b> {predictions['roberta_single']}"
    )

    # Heatmaps anzeigen
    with heatmap_output:
        heatmap_output.clear_output()
        display(widgets.HBox([
            widgets.Image(value=heatmaps["distilbert_single"], format="png", width=600, height=300),
            widgets.Image(value=heatmaps["roberta_single"], format="png", width=600, height=300)
        ]))


# 🌟 Funktion für "New Question"-Button
def on_new_question(b):
    question_text.value = get_random_question()
    answer_text.value = ""
    output_label.value = "Waiting for input..."
    with heatmap_output:
        heatmap_output.clear_output()

# 🌟 Widgets erstellen
heatmap_output = widgets.Output()

question_text = widgets.Text(value=get_random_question(), description="Question:")
answer_text = widgets.Text(description="Answer:")
predict_button = widgets.Button(description="Predict")
new_question_button = widgets.Button(description="New Question")
output_label = widgets.HTML(value="Waiting for input...")

predict_button.on_click(on_predict)
new_question_button.on_click(on_new_question)

# 🌟 UI anzeigen
display(widgets.VBox([
    question_text,
    answer_text,
    widgets.HBox([predict_button, new_question_button]),
    output_label,
    heatmap_output  # Heatmaps nebeneinander
]))


In [51]:
import json
import random
import ipywidgets as widgets
from IPython.display import display
import torch
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Laden der Datensätze
single_dataset_path = '/content/drive/My Drive/ColabData/final_single_question_data.json'
multi_dataset_path = '/content/drive/My Drive/ColabData/final_multi_question_data.json'

with open(single_dataset_path, 'r') as file:
    single_dataset = json.load(file)

with open(multi_dataset_path, 'r') as file:
    multi_dataset = json.load(file)

# Funktion zum Extrahieren der Fragen aus den Datensätzen
def extract_questions(dataset):
    return [item['question'] for item in dataset if 'question' in item]

# Fragen aus den Datensätzen extrahieren
single_questions = extract_questions(single_dataset)
multi_questions = extract_questions(multi_dataset)

# Kombinierte Fragenliste und Typzuweisung
all_questions = [(q, 'single') for q in single_questions] + [(q, 'multi') for q in multi_questions]

# Zufällige Frage auswählen
def get_random_question():
    return random.choice(all_questions)

# Pfade der Modelle
model_paths = {
    "distilbert_single": "/content/drive/My Drive/final_model",
    "roberta_single": "/content/drive/My Drive/roberta_finetuned",
    "distilbert_multi": "/content/drive/My Drive/multi_distilbert_wQ",
    "roberta_multi": "/content/drive/My Drive/multi_roberta"
}

# Laden der Modelle und Tokenizer
models = {key: AutoModelForSequenceClassification.from_pretrained(path) for key, path in model_paths.items()}
tokenizers = {key: AutoTokenizer.from_pretrained(path) for key, path in model_paths.items()}

# Laden der Label-Mappings
single_label_mapping_path = '/content/drive/My Drive/single_label_mapping.pkl'
multi_label_mapping_path = '/content/drive/My Drive/label_mapping_multi_wQ.pkl'

with open(single_label_mapping_path, 'rb') as file:
    single_label_mapping = pickle.load(file)

with open(multi_label_mapping_path, 'rb') as file:
    multi_label_mapping = pickle.load(file)

# Funktion zur Erstellung einer Heatmap
def create_heatmap(probabilities, label_mapping, title="Heatmap"):
    plt.figure(figsize=(12, 4))
    labels = [label_mapping[idx] for idx in range(len(label_mapping))]
    plt.imshow([probabilities], cmap="YlGnBu", aspect="auto")
    plt.colorbar(label="Probability")
    plt.yticks([])
    plt.xticks(range(len(probabilities)), labels, rotation=90)
    plt.title(title)
    buf = BytesIO()
    plt.savefig(buf, format="png", bbox_inches="tight")
    buf.seek(0)
    image_bytes = buf.read()
    buf.close()
    plt.close()
    return image_bytes

# Vorhersage-Funktion
def predict_answer(question, answer, question_type, multi_threshold=0.3):
    model_keys = [
        "distilbert_single" if question_type == 'single' else "distilbert_multi",
        "roberta_single" if question_type == 'single' else "roberta_multi"
    ]
    label_mapping = single_label_mapping if question_type == 'single' else multi_label_mapping

    predictions = {}
    heatmaps = {}

    for model_key in model_keys:
        tokenizer = tokenizers[model_key]
        model = models[model_key]
        inputs = tokenizer(question, answer, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
        model.eval()
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            probabilities = torch.sigmoid(logits).squeeze().tolist()

            if question_type == 'single':
                # **For single questions, return only the highest probability label**
                predicted_index = torch.argmax(torch.tensor(probabilities)).item()
                predicted_labels = [label_mapping.get(predicted_index, f"Unknown ({predicted_index})")]
            else:
                # **For multi questions, use threshold-based predictions**
                predicted_indices = [idx for idx, prob in enumerate(probabilities) if prob > multi_threshold]
                predicted_labels = [label_mapping.get(idx, f"Unknown ({idx})") for idx in predicted_indices]

            predictions[model_key] = predicted_labels if predicted_labels else "No Labels predicted"
            heatmaps[model_key] = create_heatmap(probabilities, label_mapping, title=f"{model_key} Heatmap")

    return predictions, heatmaps

# UI Widgets
question_text = widgets.Text(
    value="",
    description="Question:",
    layout=widgets.Layout(width="80%")  # Breite der Eingabefelder anpassen
)
answer_text = widgets.Text(
    description="Answer:",
    layout=widgets.Layout(width="80%")  # Breite der Eingabefelder anpassen
)

predict_button = widgets.Button(description="Predict")
new_question_button = widgets.Button(description="New Question")
output_label = widgets.HTML(value="Waiting for input...")
heatmap_output = widgets.Output()

predict_button.on_click(on_predict)
new_question_button.on_click(on_new_question)

# Initiale Frage laden
on_new_question(None)

# UI anzeigen
display(widgets.VBox([
    question_text,
    answer_text,
    widgets.HBox([predict_button, new_question_button]),
    output_label,
    heatmap_output
]))
